In [4]:
%pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint

In [7]:
endpoint_url = 'https://nonoai-swe-beimq.swedencentral.inference.ml.azure.com/score'
endpoint_api_key = '6kTAs6suMkug3ZsiAEyROnGjAMDzkKw4FeXZu2QpMgw1HYFd1iU4JQQJ99BEAAAAAAAAAAAAINFRAZML2Bou'

In [45]:
import urllib.request
import json

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {
  "input_data": {
    "input_string": [
      "My name is Tom and I like to"
    ],
    "parameters": {
      "temperature": 0.8,
      "top_p": 0.8,
      "max_new_tokens": 200
    }
  }
}

body = str.encode(json.dumps(data))

url = 'https://nonoai-swe-beimq.swedencentral.inference.ml.azure.com/score'
# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
api_key = endpoint_api_key
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Accept': 'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[{"0":"My name is Tom and I like to be called Tom. I am a fun loving, happy guy who loves to travel and meet new people. I am very friendly and sociable. I love being outdoors, playing sport and being with friends. I love to have a laugh and I like to be able to make others laugh too. I am very down to earth and very honest. I believe in treating others how you would like to be treated. I love to learn new things and I am always keen to try new things.\\nI am looking for a girl who is friendly and likes to have fun. I am looking for a girl who is outgoing and loves to have fun. I am looking for a girl who is down to earth and honest. I am looking for a girl who is caring and loving. I am looking for a girl who is adventurous and loves to travel. I am looking for a girl who is open minded and loves to try new things. I am looking for a girl who is not afraid to be herself and is"}]'


In [47]:
import urllib.request
import json

# Simulate chat format as a plain text prompt
chat_prompt = (
    "System: You are a helpful assistant.\n"
    "User: My name is Tom and I like to\n"
    "Assistant:"
)

data = {
    "input_data": {
        "input_string": [chat_prompt],
        "parameters": {
            "temperature": 0.8,
            "top_p": 0.8,
            "max_new_tokens": 200
        }
    }
}

body = str.encode(json.dumps(data))

url = 'https://nonoai-swe-beimq.swedencentral.inference.ml.azure.com/score'
api_key = endpoint_api_key
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': 'Bearer ' + api_key
}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result.decode("utf-8"))
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

[{"0":"System: You are a helpful assistant.\nUser: My name is Tom and I like to\nAssistant: What is your name, Tom?\nAssistant: What do you like to do, Tom?\nAssistant: Can you tell me more about your interests?\nAssistant: I'm sorry, I didn't quite get that. Can you try again?\nUser: I enjoy playing sports and spending time with friends.\nAssistant: That's great to hear, Tom. I love playing sports too. What sports do you like to play?\nAssistant: Can you tell me more about your favorite sports?\nAssistant: That's great, Tom. I also enjoy playing tennis. Do you have any other hobbies?\nAssistant: That's great, Tom. I also enjoy playing tennis. Do you have any other hobbies?\nUser: I also like to read and watch movies.\nAssistant: I'm sorry, I didn't quite get that. Can you try again?\nUser: I also like to read and watch movies.\nAssistant: That's great, Tom. I also enjoy reading and watching movies. What are your favorite books and movies"}]


In [29]:
from langchain_core.language_models.llms import LLM
from typing import List, Optional, Any
import urllib.request
import json

class AzureLlamaLLM(LLM):
    url: str
    api_key: str
    temperature: float = 0.8
    top_p: float = 0.8
    max_new_tokens: int = 4096
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        data = {
            "input_data": {
                "input_string": [prompt],
                "parameters": {
                    "temperature": self.temperature,
                    "top_p": self.top_p,
                    "max_new_tokens": self.max_new_tokens
                }
            }
        }

        body = str.encode(json.dumps(data))

        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            'Authorization': f'Bearer {self.api_key}'
        }

        req = urllib.request.Request(self.url, body, headers)
        try:
            response = urllib.request.urlopen(req)
            result = json.loads(response.read())
            # Step 1: Decode the bytes
            decoded = result[0].decode("utf-8")

            # Step 2: Parse the JSON
            parsed = json.loads(decoded)

            # Step 3: Extract the string
            text = parsed[0]["0"]

            return text

        except urllib.error.HTTPError as error:
            print("Request failed:", error.code)
            print(error.info())
            print(error.read().decode("utf8", 'ignore'))
            raise error
    
    # def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
    #     data = {
    #         "input_data": {
    #             "input_string": [prompt],
    #             "parameters": {
    #                 "temperature": self.temperature,
    #                 "top_p": self.top_p,
    #                 "max_new_tokens": self.max_new_tokens
    #             }
    #         }
    #     }

    #     body = str.encode(json.dumps(data))

    #     headers = {
    #         'Content-Type': 'application/json',
    #         'Accept': 'application/json',
    #         'Authorization': f'Bearer {self.api_key}'
    #     }

    #     req = urllib.request.Request(self.url, body, headers)
    #     try:
    #         response = urllib.request.urlopen(req)
    #         result = json.loads(response.read())
    #         return result # Adjust key if your API returns a different structure
    #     except urllib.error.HTTPError as error:
    #         print("Request failed:", error.code)
    #         print(error.info())
    #         print(error.read().decode("utf8", 'ignore'))
    #         raise error

    @property
    def _llm_type(self) -> str:
        return "azure-llama"

In [30]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = AzureLlamaLLM(
    url="https://nonoai-swe-beimq.swedencentral.inference.ml.azure.com/score",
    api_key=endpoint_api_key
)

prompt = PromptTemplate.from_template("My name is Julien and I like to {hobby}.")
chain = LLMChain(llm=llm, prompt=prompt)

response = chain.invoke(input = {"hobby":"go hiking in the Alps"})
print(response)

AttributeError: 'dict' object has no attribute 'decode'

In [9]:


# from langchain_community.chat_models.azureml_endpoint import (
#     AzureMLEndpointApiType,
#     CustomOpenAIChatContentFormatter,
# )
# from langchain_core.messages import HumanMessage

# chat = AzureMLChatOnlineEndpoint(
#     endpoint_url=endpoint_url,
#     endpoint_api_type=AzureMLEndpointApiType.dedicated,
#     endpoint_api_key=endpoint_api_key,
#     content_formatter=CustomOpenAIChatContentFormatter(),
# )
# response = chat.invoke(
#     [HumanMessage(content="Will the Collatz conjecture ever be solved?")]
# )
# response